In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import Runnable

# Set up your local LLM
llm = Ollama(model="mistral")

# Set up memory
memory = ConversationBufferMemory(return_messages=True)

# Define a chat-style prompt with memory
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that summarizes text, classifies sentiment, and writes tweets."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# Create the chain
chain: Runnable = (
    prompt
    | llm
    | StrOutputParser()
)

# Interactive loop
print("🧠 Context-Aware Assistant (LangChain + Mistral)")
print("Type a paragraph. Type 'exit' to quit.\n")

while True:
    user_input = input("📝 Your input: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break
    if not user_input:
        print("⚠️ Please enter some text.")
        continue

    # Inject memory into the chain
    response = chain.invoke({
        "input": user_input,
        "chat_history": memory.chat_memory.messages
    })

    # Save interaction to memory
    memory.chat_memory.add_user_message(user_input)
    memory.chat_memory.add_ai_message(response)

    print("🤖 Response:", response, "\n")